# 신경망 복습



In [1]:
import numpy as np

x = np.array([1,2,3])
print("x의 클래스 : ",x.__class__) # 클래스 이름 표시
print("x.shape : ", x.shape)
print("x.ndim : ", x.ndim)

W = np.array([[1,2,3], [4,5,6]])
print("W.shape : ",W.shape)
print("W.ndim : ", W.ndim)

x의 클래스 :  <class 'numpy.ndarray'>
x.shape :  (3,)
x.ndim :  1
W.shape :  (2, 3)
W.ndim :  2


In [2]:
A = np.array([[1,2],[3,4]])
A * 10

array([[10, 20],
       [30, 40]])

In [3]:
A = np.array([[1,2],[3,4]])
B = np.array([10,20])
A*B

array([[10, 40],
       [30, 80]])

In [4]:
# 행렬의 내적 
a = np.array([1,2,3])
b = np.array([4,5,6])
print("np.dot(a,b) : ", np.dot(a,b))

# 행렬의 곱
A = np.array([[1,2], [3,4]])
B = np.array([[5,6],[7,8]])
print("np.matmul(A,B) : ", np.matmul(A,B))

np.dot(a,b) :  32
np.matmul(A,B) :  [[19 22]
 [43 50]]


In [5]:
# 완전연결계층에 의한 변환의 미니배치 버전

W1 = np.random.randn(2,4) # 가중치
b1 = np.random.randn(4) # 편향
x = np.random.randn(10, 2) # 입력
h = np.matmul(x, W1) + b1
# 10개의 샘플 데이터 각각을 완전연결계층으로 변환
# x의 첫 번째 차원이 각 샘플 데이터에 해당한다.
# x[0]은 0번째 입력 데이터, x[1] 은 첫 번째 입력 데이터
# h[0]은 0번째 은닉층 뉴런
# b1의 덧셈은 브로드캐스트 b1의 형상은 (4,) 이지만 자동으로 (10,4)로 복제

In [6]:
def sigmoid(x):
    return 1/(1 + np.exp(-x))

In [7]:
a = sigmoid(h)

In [8]:
def sigmoid(x):
    return 1/(1 + np.exp(-x))

W1 = np.random.randn(2,4)
b1 = np.random.randn(4)
x = np.random.randn(10, 2)
W2 = np.random.randn(4,3)
b2 = np.random.randn(3)

h = np.matmul(x, W1) + b1
a = sigmoid(h)
s = np.matmul(a, W2) + b2



In [9]:
class Sigmoid:
    def __init__(self):
        self.params = []
        
    def forward(self, x):
        return 1/(1+np.exp(-x))
    

class Affine:
    def __init__(self,W,b):
        self.params = [W,b] # 가중치와 편향, 신경망이 학습될 때 수시로 갱신
        
    def forward(self, x):
        W, b = self.params
        out = np.matmul(x, W) + b
        return out

In [10]:
class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size):
        I, H, O = input_size, hidden_size, output_size
        
        #가중치와 편향 초기화
        W1 = np.random.rand(I, H) # 입력층, 은닉층
        b1 = np.random.rand(H) # 은닉층
        W2 = np.random.rand(H, O) # 은닉층, 출력층 
        b2 = np.random.rand(O) # 출력층
        
        # 계층 생성
        
        self.layers = [
            Affine(W1,b1),
            Sigmoid(),
            Affine(W2, b2)
        ]
        
        # 모든 가중치 리스트에 모으기
        self.params = []
        for layer in self.layers:
            self.params += layer.params
            
    def predict(self, x):
        for layer in self.layers:
            x = layer.forward(x)
        return x

In [11]:

x = np.random.rand(8,2)
model = TwoLayerNet(2, 4, 3)
s = model.predict(x)




In [12]:
# Repeat 노드
import numpy as np
D, N = 8,7

x = np.random.rand(1, D) # 입력
y = np.repeat(x, N, axis = 0) # 순전파
dy = np.random.rand(N, D) # 무작위 기울기
dx = np.sum(dy, axis = 0, keepdims=True) # 역전파

In [13]:
# Sum 노드

import numpy as np
D, N = 8,7

x = np.random.rand(N, D) # 입력
y = np.sum(dy, axis = 0, keepdims=True) # 역전파

dy = np.random.randn(1, D) # 무작위 기울기
dx = np.repeat(dy, N, axis = 0) # 역전파

In [14]:
# MatMul 클래스
class MatMul:
    def __init__(self, W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.x = None
        
    def forward(self, x):
        W, = self.params
        out = np.matmul(x, W)
        self.x = x
        return out
    
    def backward(self, dout):
        W, = self.params
        dx = np.matmul(dout, W.T)
        dW = np.matmul(self.x.T, dout)
        self.grads[0][...] = dW
        return dx

In [15]:
# Sigmoid 계층

class Sigmoid:
    def __init__(self):
        self.params, self.grads = [],[]
        self.out = None
        
    def forward(self, x):
        out = 1/(1+np.exp(-x))
        self.out = out
        return out
    
    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out
        return dx

In [16]:
# Affine 계층 그래프

class Affine:
    def __init__(self, W, b):
        self.params = [W,b]
        self.grads = [np.zeros_like(W), np.zeros_like(b)]
        self.x = None
        
    def forward(self, x):
        W, b = self.params
        out = np.matmul(x, W) + b
        self.x = x
        return x
    
    def backward(self, dout):
        W, b = self.params
        dx = np.matmul(dout, W.T)
        dW = np.matmul(self.x.T, dout)
        db = np.sum(dout, axis = 0)
        
        self.grads[0][...] = dW
        self.grads[1][...] = db
        return dx

In [17]:
class SGD:
    def __init__(self, lr=0.01):
        self.lr = lr
        
    def update(self, params, grads): # params 신경망 가중치, grads 기울기
        for i in range(len(params)):
            params[i] -= self.lr * grads[i]

In [2]:
import cupy as cp
x = cp.arange(6).reshape(2, 3).astype('f')
print("x : ", x)
print("x.sum : ",x.sum(axis=1))

x :  [[0. 1. 2.]
 [3. 4. 5.]]
x.sum :  [ 3. 12.]
